In [218]:
from datetime import timedelta
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib.dates as mdates
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [219]:
df = pd.read_csv(
    r'/Users/yotamhadari/Desktop/SOLIO/Group_1_SOLIO_final.csv',
    index_col=[0],
    #parse_dates=True,
    #dayfirst=True,
)
df.tail()

# df["Week"] = np.nan
# week_count = 1
# day_count = 1
# for n in range(0, len(df)):
#     date_bool = (
#         df.index.get_level_values(1)[n] - df.index.get_level_values(1)[n - 1]
#     ) >= timedelta(3)
#     match date_bool:
#         case True:  ## This Section handles creating a "week in trial" column by using the dates given a part of the index
#             week_count += 1  ## Sunday is the first day of the week!!!!
#             df.iloc[n, -1] = week_count
#         case False:
#             df.iloc[n, -1] = week_count

,Date,Pain VAS,Qualitative pain,Day to day,Lifting weights,Walking,Sitting,Standing,Sleeping,Sex life,Social life,Travel
User ID,,,,,,,,,,,,
21,10/07/2022,3,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,11/07/2022,4,NaN,NaN,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
21,12/07/2022,4,NaN,NaN,NaN,NaN,2.0,5.0,NaN,NaN,NaN,NaN
21,13/07/2022,4,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN
21,14/07/2022,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,4.0


In [220]:
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
df['Week'] = df['Date'].dt.isocalendar().week
df['Day_of_week'] = df['Date'].dt.day_name()
df.loc[df['Day_of_week'] == 'Sunday', 'Week'] = df.loc[df['Day_of_week'] == 'Sunday', 'Week'] + 1
first_week = min(df['Week'].drop_duplicates())
df['Week'] = df['Week'] - first_week + 1

In [225]:
df1 = df.loc[df.index==1]
fig = make_subplots(rows=3, cols=3)
for i, col in enumerate(df.columns[2:-2]):
    fig.add_trace(go.Bar(x=df1['Week'], y=df[col], name=col), 1,1) 

fig.update_layout(margin_r=10)
fig.show()

In [233]:
fig = go.Figure([go.Bar(x=df1['Week'], y=df[col], name=col, text=df1['Date'].dt.date) for col in df1.columns[2:-2]])
fig.show()

In [229]:
df1.head()

,Date,Pain VAS,Qualitative pain,Day to day,Lifting weights,Walking,Sitting,Standing,Sleeping,Sex life,Social life,Travel,Week,Day_of_week
User ID,,,,,,,,,,,,,,
1,2022-07-17,0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,Sunday
1,2022-07-18,0,NaN,NaN,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2,Monday
1,2022-07-19,1,NaN,NaN,NaN,NaN,3.0,1.0,NaN,NaN,NaN,NaN,2,Tuesday
1,2022-07-20,2,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN,NaN,2,Wednesday
1,2022-07-21,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,2,Thursday
